In [1]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer

model_name = "dkleczek/bert-base-polish-uncased-v1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

C:\Users\przem\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at dkleczek/bert-base-polish-uncased-v1 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
from helpers import load_poleval_in_answers

test_a_df = load_poleval_in_answers("data_poleval/test-A")
test_a_df.head()

,question,answers
0,Czy poeta Lucjan Rydel tworzył także sztuki te...,[tak]
1,W którym państwie została ogłoszona „Deklaracj...,[we Francji]
2,Która kawa zawiera alkohol: po turecku czy po ...,[po irlandzku]
3,W którym mieście zmarł Sławomir Mrożek?,[w Nicei]
4,Jak nazywał się autor powieści „Wierna rzeka”?,[Stefan Żeromski]


In [ ]:
from bert_helpers import generate_context_gpt
test_a_df["context"] = test_a_df["question"].apply(generate_context_gpt)
test_a_df.head()
# test_a_df.to_csv("llm_generated_context_short.csv", index=False)

In [3]:
import pandas as pd

test_a_df = pd.read_csv(r"llm_generated_context_short.csv")
test_a_df.head()

,question,answers,context
0,Czy poeta Lucjan Rydel tworzył także sztuki te...,['tak'],W literaturze Polski odpowiedź jest tak.
1,W którym państwie została ogłoszona „Deklaracj...,['we Francji'],Zdarzenie miało miejsce we Francji.
2,Która kawa zawiera alkohol: po turecku czy po ...,['po irlandzku'],Słynna osoba lub rzecz to po irlandzku.
3,W którym mieście zmarł Sławomir Mrożek?,['w Nicei'],Zdarzenie miało miejsce w Nicei.
4,Jak nazywał się autor powieści „Wierna rzeka”?,['Stefan Żeromski'],Słynna osoba lub rzecz to Stefan Żeromski.


In [4]:
contain_ratio_all = sum(test_a_df.apply(lambda r: any(str(a).lower() in str(r["context"]).lower() for a in r["answers"]), axis=1)) / len(test_a_df)
print("Answer-in-context ratio (all):", contain_ratio_all)

Answer-in-context ratio (all): 1.0


In [6]:
from bert_helpers import answer_in_context_row
from sklearn.model_selection import train_test_split


train_df, eval_df = train_test_split(test_a_df, test_size=0.2, random_state=42)
train_df = train_df[train_df.apply(answer_in_context_row, axis=1)].reset_index(drop=True)
eval_df  = eval_df[eval_df.apply(answer_in_context_row, axis=1)].reset_index(drop=True)
print("After filtering: train size =", len(train_df), "eval size =", len(eval_df))

After filtering: train size = 2000 eval size = 500


In [7]:
from bert_helpers import to_squad_style

train_dataset = to_squad_style(train_df)
eval_dataset  = to_squad_style(eval_df)
print("Train dataset examples:", len(train_dataset), "Eval dataset examples:", len(eval_dataset))


Train dataset examples: 2000 Eval dataset examples: 500


In [9]:
tokenizer = AutoTokenizer.from_pretrained("dkleczek/bert-base-polish-uncased-v1")

def preprocess(examples):
    questions = [q.strip() for q in examples["question"]]
    contexts = examples["context"]
    tokenized = tokenizer(
        questions,
        contexts,
        truncation="only_second",
        padding="max_length",
        max_length=512,
        return_offsets_mapping=True
    )
    start_positions = []
    end_positions = []
    for i in range(len(questions)):
        offsets = tokenized["offset_mapping"][i]
        answer = examples["answers"][i]
        if isinstance(answer, dict):
            answer_text = answer["text"][0]
            start_char = answer["answer_start"][0]
        else:
            answer_text = str(answer[0])
            start_char = contexts[i].find(answer_text)
        end_char = start_char + len(answer_text)
        start_token = tokenized.char_to_token(i, start_char)
        end_token = tokenized.char_to_token(i, end_char - 1) if end_char > 0 else None
        if start_token is None or end_token is None:
            start_positions.append(-100)
            end_positions.append(-100)
        else:
            start_positions.append(start_token)
            end_positions.append(end_token)

    tokenized["start_positions"] = start_positions
    tokenized["end_positions"] = end_positions
    tokenized.pop("offset_mapping", None)
    return tokenized

train_enc = train_dataset.map(preprocess, batched=True, remove_columns=train_dataset.column_names)
eval_enc  = eval_dataset.map(preprocess, batched=True, remove_columns=eval_dataset.column_names)

Map: 100%|██████████| 500/500 [00:00<00:00, 5405.02 examples/s]


In [14]:
training_args = TrainingArguments(
    output_dir="./qa_poleval",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    save_total_limit=2,
    logging_steps=50,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    eval_steps=300,
    save_steps=300,
    save_strategy="steps",
)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_enc,
    eval_dataset=eval_enc,
)

trainer.train()

Step,Training Loss
50,2.828200
100,2.160400
150,2.050400
200,1.965400
250,1.844900
300,1.506600
350,1.432900
400,1.445200
450,1.457300
500,1.429200


C:\Users\przem\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=750, training_loss=1.5353456115722657, metrics={'train_runtime': 7240.765, 'train_samples_per_second': 0.829, 'train_steps_per_second': 0.104, 'total_flos': 1567780540416000.0, 'train_loss': 1.5353456115722657, 'epoch': 3.0})

In [16]:
from bert_helpers import poleval_accuracy_strict
from transformers import pipeline
import torch


qa_pipe = pipeline(
    "question-answering",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,
    handle_impossible_answer=False,
    max_answer_len=30,
    topk=5
)

acc = poleval_accuracy_strict(eval_df, qa_pipe)
print(f"Poleval-style accuracy: {acc:.4f}")

Device set to use cpu
C:\Users\przem\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\pipelines\question_answering.py:339: UserWarning: topk parameter is deprecated, use top_k instead
  warnings.warn("topk parameter is deprecated, use top_k instead", UserWarning)


Poleval-style accuracy: 0.0020


In [19]:
from bert_helpers import poleval_accuracy_context

acc = poleval_accuracy_context(eval_df, qa_pipe)
print(f"Poleval-style accuracy: {acc:.4f}")

Poleval-style accuracy: 0.8580


In [18]:
for row in eval_df.sample(25).itertuples():
    out = qa_pipe(
        question=row.question,
        context=row.context,
        topk=5
    )
    best = out[0]
    print("Q:", row.question)
    print("PRED:", best["answer"])
    print("GOLD:", row.answers)
    print("-"*5)

Q: W którym mieście w Wielkopolsce miał miejsce w latach 1902–03 strajk uczniów przeciw germanizacji szkoły?
PRED: Wrześni
GOLD: ['we Wrześni']
-----
Q: Jakie imię nosi archanioł opisany w Księdze Daniela jako opiekun narodu izraelskiego?
PRED: lub rzecz to
GOLD: ['Michał']
-----
Q: W którym roku Chorwacja przystąpiła do Unii Europejskiej?
PRED: miało
GOLD: ['w 2013']
-----
Q: Premiera „Dziadów” w reżyserii Wyspiańskiego miała miejsce w XIX czy XX wieku?
PRED: miało
GOLD: ['w XX', '20']
-----
Q: Jak z niemieckiego nazywa się myśliwska broń łamana o trzech lufach?
PRED: to dryling
GOLD: ['dryling', 'trójlufka', 'trojak']
-----
Q: Jakiej narodowości był Ferenc Molnar, autor „Chłopców z Placu Broni”?
PRED: Węgrem
GOLD: ['był Węgrem']
-----
Q: Który bohater „Lalki”, autor pamiętnika, brał udział w rewolucji węgierskiej 1848 r.?
PRED: to Ignacy Rzecki
GOLD: ['Ignacy Rzecki']
-----
Q: Jak z łaciny nazywamy okres po studiach prawniczych przygotowujący osoby do pracy sędziego, adwokata?
PRED: 